# Minilab 2

<b>Class:</b> MSDS 7331 Data Mining
<br> <b>Dataset:</b> Belk Endowment Educational Attainment Data 

<h1 style="font-size:150%;"> Teammates </h1>
Maryam Shahini
<br> Murtada Shubbar
<br> Michael Toolin
<br> Steven Millett

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import math
import re
import statistics
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from IPython.display import display, HTML

%matplotlib inline

In [7]:
#
# The 2017 Public Schools Machine Learning Date Set is being used throughout this analysis.  The _ML suffix is removed to less name space size
#
# Load Full Public School Data Frames for each year

school_data = pd.read_csv('../Data/2017/machine Learning Datasets/PublicSchools2017_ML.csv', low_memory=False)

# Create Models
Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe. 

In [21]:
#split data into X and y dataframes

SPG_Grade_col = school_data.filter(regex=('^SPG\WGrade')).columns
y = school_data[SPG_Grade_col].apply(lambda row:'A' if row.any()!=1 else 
                                 row[0]*'A+NG'+row[1]*'B'+row[2]*'C'+row[3]*'D'+row[4]*'F'+row[5]*'I',axis=1)

X = school_data[school_data.columns.drop(list(school_data.filter(regex='^SPG\WGrade|unit_code')))]




In [22]:
#split X and y into test and train sets.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [23]:
#applied a scaling procedure to scale the size of variables in the x dataframe

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scale = sc.fit_transform(X_train)
X_test_scale = sc.transform(X_test)

In [24]:
#Initializing the Logistic Regression function to a classifier variable
classifier = LogisticRegression()

#Fitting the Logistic Regression Classifier to the original dataset and then predicting.
classifier.fit(X_train, y_train)
y_pred_Log_Reg = classifier.predict(X_test)

#Fitting the Logistic Regression Classifier to the scaled dataset and then predicting.
classifier.fit(X_train_scale, y_train)
y_pred_Log_Reg_scale = classifier.predict(X_test_scale)

Below we have the confusion matrix and accuracy score for SVM without scaling. Based on the accuracy score it is evident that Logistic Regression handles non-standardized variables relatively well.

In [25]:
# Created a confusion matrix for the 
cm_Log_Reg = confusion_matrix(y_test, y_pred_Log_Reg)

print(cm_Log_Reg)
print(accuracy_score(y_test, y_pred_Log_Reg))

[[ 18   5   6   0   0   0   1]
 [  0   8  10   0   0   0   0]
 [  5   4 106  22   0   0   0]
 [  0   1  15 159  10   0   0]
 [  0   0   0  18  66   5   0]
 [  0   0   0   0   9  11   0]
 [  0   0   0   0   0   0  10]]
0.7730061349693251


Below we have the confusion matrix and accuracy score for logistic regression with scaling. Based on the accuracy score it is obvious that the logistic model is negatively affected by the different scales of the dataset, but the scaled model does not dramatically improve the fit of the model.

In [26]:
cm_Log_Reg_scale = confusion_matrix(y_test, y_pred_Log_Reg_scale)

print(cm_Log_Reg_scale)
print(accuracy_score(y_test, y_pred_Log_Reg_scale))

[[ 24   1   5   0   0   0   0]
 [  2  16   0   0   0   0   0]
 [  5   0 116  16   0   0   0]
 [  1   0  20 157   7   0   0]
 [  0   0   2  17  66   4   0]
 [  1   0   0   0   7  12   0]
 [  0   0   0   0   0   0  10]]
0.820040899795501


In [27]:
#Initializing the SVM function to a classifier variable
classifier = svm.SVC()

#Fitting the SVM Classifier to the original dataset and then predicting.
classifier.fit(X_train, y_train)
y_pred_svm = classifier.predict(X_test)

#Fitting the SVM Classifier to the scaled dataset and then predicting.
classifier.fit(X_train_scale, y_train)
y_pred_svm_scale = classifier.predict(X_test_scale)

Below we have the confusion matrix and accuracy score for SVM without scaling. Based on the accuracy score it is obvious that SVM is heavily influenced by non-standardized features.

In [28]:
#Assigning a confusion matrix with the original dataset.
cm_svm = confusion_matrix(y_test, y_pred_svm)

print(cm_svm)
print(accuracy_score(y_test, y_pred_svm))

[[  0   0   0  30   0   0   0]
 [  0   0   0  18   0   0   0]
 [  0   0   0 137   0   0   0]
 [  0   0   0 185   0   0   0]
 [  0   0   0  89   0   0   0]
 [  0   0   0  20   0   0   0]
 [  0   0   0  10   0   0   0]]
0.3783231083844581


Below we have the confusion matrix and accuracy score for SVM with scaling. Based on the accuracy score it is obvious that the SVM model is negatively affected by the different scales of the dataset. With the use of the standard scaler function we find that SVM is a superior model.

In [29]:
#Assigning a confusion matrix with the scaled dataset.
cm_svm_scale = confusion_matrix(y_test, y_pred_svm_scale)

print(cm_svm_scale)
print(accuracy_score(y_test, y_pred_svm_scale))

[[ 26   0   3   1   0   0   0]
 [  1  17   0   0   0   0   0]
 [  4   0 117  15   1   0   0]
 [  1   0  10 168   6   0   0]
 [  0   0   0  15  73   1   0]
 [  1   0   0   0   7  12   0]
 [  0   0   0   0   0   0  10]]
0.8650306748466258


# Model Advantages
Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.

In [ ]:
Looking at the accuracy of the two models there are a couple of obvious observations.

1. SVM is negatively to non-standardized observations.
2. Logistic Regression ...

# Interpret Feature Importance
Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?


# Interpret Support Vectors
Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model— then analyze the support vectors from the subsampled dataset.